In [296]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder

from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

Loading BokehJS ...

In [7]:
df = gpd.read_file('/Users/dianaow/Documents/singapore_stories/parliament/data/electoral-boundary-2020.json')
#Find the center point
df['Center_point'] = df['geometry'].centroid
#Extract lat and lon from the centerpoint
df["lat"] = df.Center_point.map(lambda p: p.x)
df["long"] = df.Center_point.map(lambda p: p.y)

/Users/dianaow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df.head()

,Name,description,altitudeMode,id,FID,ED_DESC,Field_1,geometry,Center_point,lat,long
0,TANJONG PAGAR,None,clampToGround,TP,2,TANJONG PAGAR,TANJONG PAGAR,"MULTIPOLYGON (((103.84580 1.26163, 103.84640 1...",POINT (103.82306 1.30169),103.823059,1.301685
1,JALAN BESAR,None,clampToGround,JB,3,JALAN BESAR,JALAN BESAR,"MULTIPOLYGON (((103.87383 1.28217, 103.87385 1...",POINT (103.85724 1.30020),103.857243,1.300203
2,PIONEER,None,clampToGround,PI,5,PIONEER,PIONEER,"MULTIPOLYGON (((103.70830 1.34003, 103.70895 1...",POINT (103.70086 1.33820),103.700861,1.338201
3,YUHUA,None,clampToGround,YH,7,YUHUA,YUHUA,"MULTIPOLYGON (((103.73730 1.34596, 103.73681 1...",POINT (103.73820 1.33945),103.738196,1.339454
4,MARYMOUNT,None,clampToGround,MR,12,MARYMOUNT,MARYMOUNT,"MULTIPOLYGON (((103.85226 1.36033, 103.85194 1...",POINT (103.84275 1.35775),103.842752,1.357749


In [239]:
df_ward = gpd.read_file('/Users/dianaow/Documents/singapore_stories/parliament/data/GE2020.json')
#Find the center point
df_ward['Center_point'] = df_ward['geometry'].centroid
#Extract lat and lon from the centerpoint
df_ward["lat"] = df_ward.Center_point.map(lambda p: p.x)
df_ward["long"] = df_ward.Center_point.map(lambda p: p.y)

/Users/dianaow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [240]:
df_ward.head()

,Name,Description,geometry,Center_point,lat,long
0,MOUNTBATTEN SMC,GRC/SMC: Single Member Constituency<br>Member ...,"POLYGON Z ((103.90455 1.29702 0.00000, 103.903...",POINT (103.88395 1.29795),103.883951,1.297949
1,Marine Parade_3,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.91000 1.30547 0.00000, 103.908...",POINT (103.90716 1.30685),103.907155,1.306847
2,Geylang Serai_2,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.89621 1.33776 0.00000, 103.895...",POINT (103.89268 1.32820),103.892684,1.328197
3,Braddell Heights_1,GRC/SMC: MARINE PARADE GRC<br>Member of Parlia...,"POLYGON Z ((103.88018 1.34885 0.00000, 103.879...",POINT (103.87097 1.34847),103.870966,1.348472
4,MACPHERSON SMC,GRC/SMC: Single Member Constituency<br>Member ...,"POLYGON Z ((103.88910 1.33247 0.00000, 103.888...",POINT (103.88609 1.32399),103.886086,1.323992


In [272]:
wards = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - Wards.csv")
constituencies = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - Consituencies.csv")
designation = pd.read_csv("/Users/dianaow/Documents/singapore_stories/parliament/data/Singapore Members of Parliament - 14th_Parliament.csv")

In [273]:
constituencies['Division'] = constituencies['Division'].str.upper() 
constituencies = pd.merge(constituencies, df.rename(columns={"Name": 'Division'})\
                          [['Division', 'lat', 'long']], on='Division', how='left')

constituencies = pd.merge(constituencies, wards, on='Name', how='left')
constituencies = pd.merge(constituencies, df_ward.rename(columns={"Name": 'Ward', 'lat': 'lat_ward', 'long': 'lon_ward'})\
                          [['Ward', 'lat_ward', 'lon_ward']], on='Ward', how='left')

constituencies['Ward'] = constituencies['Ward'].apply(lambda x: x.split('_')[0])
constituencies['newName'] = constituencies['Name'].apply(lambda x : x.replace(' ', '_'))

constituencies = pd.merge(constituencies, designation.rename(columns={'Type': 'Title'}), on='Name', how='left')

In [274]:
constituencies['Title_1'] = constituencies['Title_1'].replace('Deputy Prime Minister', 'Senior Minister')
constituencies['Title_1'] = constituencies['Title_1'].replace('Minister i/c of Social Services Integration', 'Minister')

constituencies['Title_2'] = constituencies['Title_1'].replace('Deputy Prime Minister', 'Senior Minister')
constituencies['Title_2'] = constituencies['Title_1'].replace('Senior Minister of State', 'Minister of State')
constituencies['Title_2'] = constituencies['Title_1'].replace('Senior Parliamentary Secretary', 'Parliamentary Secretaries')

In [275]:
for i,row in constituencies.iterrows():
    if(constituencies.loc[i, "Age"] <= 35):
        constituencies.loc[i, "Age_group"] = 'Less than 35'
    elif((constituencies.loc[i, "Age"] > 35) & (constituencies.loc[i, "Age"] <= 40)):
        constituencies.loc[i, "Age_group"] = '35 - 40'
    elif((constituencies.loc[i, "Age"] > 40) & (constituencies.loc[i, "Age"] <= 45)):
        constituencies.loc[i, "Age_group"] = '40 - 45'
    elif((constituencies.loc[i, "Age"] > 45) & (constituencies.loc[i, "Age"] <= 50)):
        constituencies.loc[i, "Age_group"] = '45 - 50'
    elif((constituencies.loc[i, "Age"] > 50) & (constituencies.loc[i, "Age"] <= 55)):
        constituencies.loc[i, "Age_group"] = '50 - 55'
    elif((constituencies.loc[i, "Age"] > 55) & (constituencies.loc[i, "Age"] <= 60)):
        constituencies.loc[i, "Age_group"] = '55 - 60'
    elif((constituencies.loc[i, "Age"] > 60) & (constituencies.loc[i, "Age"] <= 65)):
        constituencies.loc[i, "Age_group"] = '60 - 65'
    elif(constituencies.loc[i, "Age"] > 65):
        constituencies.loc[i, "Age_group"] = 'More than 65'

In [276]:
constituencies.columns

Index(['Division', 'Type', 'Name', 'lat', 'long', 'Ward', 'lat_ward',
       'lon_ward', 'newName', 'Party', 'Age', 'Years_In_Politics', 'Ethnicity',
       'Gender', 'Title', 'Parliament', 'Type.1', 'Ministry_1', 'Ministry_2',
       'Ministry_3', 'Title_1', 'Title_2', 'Title_3', 'Defence',
       'Foreign Affairs', 'Health', 'Transport',
       'Sustainability and the Environment', 'Social & Family Development',
       'Communications & Information', 'Culture, Community and Youth',
       'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law',
       'National Development', 'Home Affairs', 'Age_group'],
      dtype='object')

In [278]:
constituencies.isnull().sum()

Division                               0
Type                                   1
Name                                   0
lat                                    0
long                                   0
Ward                                   0
lat_ward                               0
lon_ward                               0
newName                                0
Party                                  0
Age                                    0
Years_In_Politics                      0
Ethnicity                              0
Gender                                 0
Title                                  0
Parliament                             0
Type.1                                 0
Ministry_1                            56
Ministry_2                            69
Ministry_3                            90
Title_1                                0
Title_2                                0
Title_3                               90
Defence                                0
Foreign Affairs 

In [280]:
X_subset = constituencies[['Division', 'Party', 'Age_group', 'Years_In_Politics', 'Ethnicity', 'Gender', 'Title_1', \
                           'Defence', 'Foreign Affairs', 'Health', 'Transport', 'Sustainability and the Environment', \
                           'Social & Family Development','Communications & Information', 'Culture, Community and Youth',\
                           'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law', \
                           'National Development', 'Home Affairs' ]]

In [281]:
enc = OneHotEncoder(handle_unknown='ignore')
X_enc = enc.fit_transform(X_subset).toarray()
enc_cols = enc.get_feature_names(X_subset.columns)
df_X = pd.DataFrame(X_enc, columns=enc_cols)

In [282]:
for i in df_X.columns:
    print(i)

Division_ALJUNIED
Division_ANG MO KIO
Division_BISHAN-TOA PAYOH
Division_BUKIT BATOK
Division_BUKIT PANJANG
Division_CHUA CHU KANG
Division_EAST COAST
Division_HOLLAND-BUKIT TIMAH
Division_HONG KAH NORTH
Division_HOUGANG
Division_JALAN BESAR
Division_JURONG
Division_KEBUN BARU
Division_MACPHERSON
Division_MARINE PARADE
Division_MARSILING-YEW TEE
Division_MARYMOUNT
Division_MOUNTBATTEN
Division_NEE SOON
Division_PASIR RIS-PUNGGOL
Division_PIONEER
Division_POTONG PASIR
Division_PUNGGOL WEST
Division_RADIN MAS
Division_SEMBAWANG
Division_SENGKANG
Division_TAMPINES
Division_TANJONG PAGAR
Division_WEST COAST
Division_YIO CHU KANG
Division_YUHUA
Party_People's Action Party
Party_Workers' Party
Age_group_35 - 40
Age_group_40 - 45
Age_group_45 - 50
Age_group_50 - 55
Age_group_55 - 60
Age_group_60 - 65
Age_group_Less than 35
Age_group_More than 65
Years_In_Politics_0
Years_In_Politics_4
Years_In_Politics_5
Years_In_Politics_9
Years_In_Politics_14
Years_In_Politics_19
Years_In_Politics_23
Years_

In [302]:
from sklearn.manifold import TSNE

RS = 123
tsne_features = TSNE(random_state=RS, perplexity=30).fit_transform(X_enc)

# Make X, Y columns 
constituencies['X'] = tsne_features[:, 0]
constituencies['Y'] = tsne_features[:, 1]

In [303]:
# Make a source and a scatter plot  
source = ColumnDataSource(constituencies)

plot = figure(x_axis_label = 'T-SNE 1', 
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)

plot.circle(x = "X", 
    y = "Y", 
    source = source, 
    size = 10, color = '#FF7373', alpha = .5)

GlyphRenderer(id='2896', ...)

In [304]:
# Create a HoverTool object
hover = HoverTool(tooltips = [('Name', '@Name')])
plot.add_tools(hover)

In [305]:
# Plot the map
show(plot)

In [108]:
# Print the ingredients of two similar cosmetics
p1 = constituencies[constituencies['Name'] == "Baey Yam Keng"]
p2 = constituencies[constituencies['Name'] == "Ong Ye Kung"]

# Display each item's data and ingredients
display(p1)
display(p2)

,Division,Type,Name,lat,long,newName,Party,Age,Years_In_Politics,Ethnicity,...,Education,Trade and Industry,Manpower,Finance,PMO,Law,National Development,Home Affairs,X,Y
4,TAMPINES,GRC,Baey Yam Keng,103.941521,1.359438,Baey_Yam_Keng,People's Action Party,49,14,Chinese,...,0,0,0,0,0,0,0,0,2.104645,-2.349958


,Division,Type,Name,lat,long,newName,Party,Age,Years_In_Politics,Ethnicity,...,Education,Trade and Industry,Manpower,Finance,PMO,Law,National Development,Home Affairs,X,Y
63,SEMBAWANG,GRC,Ong Ye Kung,103.809081,1.451348,Ong_Ye_Kung,People's Action Party,50,5,Chinese,...,0,0,0,0,0,0,0,0,2.254745,-2.375359


In [110]:
# Print the ingredients of two similar cosmetics
p1 = constituencies[constituencies['Name'] == "Sun Xueling"]
p2 = constituencies[constituencies['Name'] == "Gan Siow Huang"]

# Display each item's data and ingredients
display(p1)
display(p2)

,Division,Type,Name,lat,long,newName,Party,Age,Years_In_Politics,Ethnicity,...,Education,Trade and Industry,Manpower,Finance,PMO,Law,National Development,Home Affairs,X,Y
76,PUNGGOL WEST,SMC,Sun Xueling,103.895945,1.403923,Sun_Xueling,People's Action Party,41,5,Chinese,...,1,0,0,0,0,0,0,0,5.850972,-7.865757


,Division,Type,Name,lat,long,newName,Party,Age,Years_In_Politics,Ethnicity,...,Education,Trade and Industry,Manpower,Finance,PMO,Law,National Development,Home Affairs,X,Y
25,MARYMOUNT,SMC,Gan Siow Huang,103.842752,1.357749,Gan_Siow_Huang,People's Action Party,45,0,Chinese,...,1,0,1,0,0,0,0,0,5.809743,-7.848806


In [196]:
X_subset1 = constituencies[['Division', 'Party', 'Age_group', 'Years_In_Politics', 'Ethnicity', 'Gender', 'Title_2', \
                           'Defence', 'Foreign Affairs', 'Health', 'Transport', 'Sustainability and the Environment', \
                           'Social & Family Development','Communications & Information', 'Culture, Community and Youth',\
                           'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law', \
                           'National Development', 'Home Affairs' ]]

enc = OneHotEncoder(handle_unknown='ignore')
X_enc1 = enc.fit_transform(X_subset1).toarray()
enc_cols1 = enc.get_feature_names(X_subset1.columns)
df_X1 = pd.DataFrame(X_enc1, columns=enc_cols1)

tsne_features1 = TSNE(random_state=RS).fit_transform(X_enc1)

# Make X, Y columns 
constituencies['X1'] = tsne_features1[:, 0]
constituencies['Y1'] = tsne_features1[:, 1]

In [197]:
# Make a source and a scatter plot  
source = ColumnDataSource(constituencies)

plot = figure(x_axis_label = 'T-SNE 1', 
              y_axis_label = 'T-SNE 2', 
              width = 500, height = 400)

plot.circle(x = "X1", 
    y = "Y1", 
    source = source, 
    size = 10, color = '#FF7373', alpha = .5)

GlyphRenderer(id='2049', ...)

In [198]:
# Create a HoverTool object
hover = HoverTool(tooltips = [('Name', '@Name')])
plot.add_tools(hover)

In [ ]:
# Plot the map
show(plot)

In [289]:
constituencies.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/mps_constituencies.csv')

In [310]:
constituencies['Title_1'].unique()

array(['Member of Parliament', 'Minister of State',
       'Senior Minister of State', 'Senior Parliamentary Secretary',
       'Minister', 'Parliamentary Secretaries', 'Senior Minister',
       'Prime Minister'], dtype=object)

In [316]:
constituencies[constituencies['Title_1'] == 'Parliamentary Secretaries']

,Division,Type,Name,lat,long,Ward,lat_ward,lon_ward,newName,Party,...,Trade and Industry,Manpower,Finance,PMO,Law,National Development,Home Affairs,Age_group,X,Y
22,TANJONG PAGAR,GRC,Eric Chua,103.823059,1.301685,Queenstown,103.804782,1.291720,Eric_Chua,People's Action Party,...,0,0,0,0,0,0,0,40 - 45,2.87824,4.021327
69,JURONG,GRC,Rahayu Mahzam,103.745311,1.335105,Bukit Batok East,103.757631,1.337088,Rahayu_Mahzam,People's Action Party,...,0,0,0,0,0,0,0,35 - 40,-2.25323,7.951070


In [355]:
df = pd.read_csv("./data/Singapore Members of Parliament - Members_Parliament_Long.csv")

In [356]:
df.head()

,Name,Party,Ethnicity,Gender,Type,Parliament
0,A. Rahim Ishak,People's Action Party,Malay,Male,MP,1
1,A.P. Rajah,Independent,Malay,Male,MP,1
2,Ang Nam Piau,People's Action Party,Chinese,Male,MP,1
3,"Bani, S.T.",Barisan Sosialis,Indian,Male,MP,1
4,"Barker, E.W.",People's Action Party,Eurasian,Male,MP,1


In [357]:
df = df[df['Type'] == 'MP']

In [380]:
df_ethnicity = df.groupby(['Parliament', 'Ethnicity'])['Name'].count().reset_index()
df_ethnicity.rename(columns={'Name': 'value'}, inplace=True)
df_ethnicity_pt =  pd.pivot_table(df_ethnicity, index='Parliament', columns='Ethnicity', values='value', fill_value=0).reset_index(drop=True)
df_ethnicity_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/ethnicity_timeline_PT.csv')
df_ethnicity.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/ethnicity_timeline.csv')

In [381]:
df_gender = df.groupby(['Parliament', 'Gender'])['Name'].count().reset_index()
df_gender.rename(columns={'Name': 'value'}, inplace=True)
df_gender_pt =  pd.pivot_table(df_gender, index='Parliament', columns='Gender', values='value', fill_value=0).reset_index()
df_gender_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/gender_timeline_PT.csv')
df_gender.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/gender_timeline.csv')

In [382]:
df_party = df.groupby(['Parliament', 'Party'])['Name'].count().reset_index()
df_party.rename(columns={'Name': 'value'}, inplace=True)
df_party_pt =  pd.pivot_table(df_party, index='Parliament', columns='Party', values='value', fill_value=0).reset_index()
df_party_pt.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/party_timeline_PT.csv')
df_party.to_csv('/Users/dianaow/Documents/singapore_stories/parliament/data/party_timeline.csv')

In [389]:
parties = ["People's Action Party", "Workers' Party", 'Barisan Sosialis', 'Independent', 'Singapore Democratic Alliance','Singapore Democratic Party']

In [394]:
df_party['Party'] = pd.Categorical(df_party['Party'], categories=parties, ordered=False)

In [404]:
constituencies['code'] = constituencies['Defence'].astype(str) + constituencies['Foreign Affairs'].astype(str) + \
constituencies['Health'].astype(str) + constituencies['Transport'].astype(str) + \
constituencies['Sustainability and the Environment'].astype(str) + constituencies['Social & Family Development'].astype(str) + \
constituencies['Communications & Information'].astype(str) + constituencies['Culture, Community and Youth'].astype(str) + \
constituencies['Education'].astype(str) + constituencies['Trade and Industry'].astype(str) + \
constituencies['Manpower'].astype(str) + constituencies['Finance'].astype(str) + \
constituencies['PMO'].astype(str) + constituencies['Law'].astype(str) + \
constituencies['National Development'].astype(str) + constituencies['Home Affairs'].astype(str)

In [407]:
constituencies['code'].value_counts()

0000000000000000    56
0010000000000000     3
0000000000001000     3
0001000000000000     2
1000000000000000     2
0000000101000000     2
0010001000000000     1
1000000000100000     1
0000000000100001     1
0000000000011000     1
0000000000000101     1
0000010010000000     1
0000010100000000     1
0101000000000000     1
0000001000000010     1
0000000001101000     1
0000000001000000     1
0000100000000000     1
0000100000000001     1
0000000000011010     1
0000000010100000     1
0001100000000000     1
0000000010010000     1
0000001001000000     1
0000010000000010     1
0100000010001000     1
0100000000000000     1
0000000100000100     1
0010010000000000     1
0000000000001010     1
0000000000000011     1
Name: code, dtype: int64

In [403]:
constituencies.columns

Index(['Division', 'Type', 'Name', 'lat', 'long', 'Ward', 'lat_ward',
       'lon_ward', 'newName', 'Party', 'Age', 'Years_In_Politics', 'Ethnicity',
       'Gender', 'Title', 'Parliament', 'Type.1', 'Ministry_1', 'Ministry_2',
       'Ministry_3', 'Title_1', 'Title_2', 'Title_3', 'Defence',
       'Foreign Affairs', 'Health', 'Transport',
       'Sustainability and the Environment', 'Social & Family Development',
       'Communications & Information', 'Culture, Community and Youth',
       'Education', 'Trade and Industry', 'Manpower', 'Finance', 'PMO', 'Law',
       'National Development', 'Home Affairs', 'Age_group', 'X', 'Y', 'code'],
      dtype='object')

In [408]:
constituencies

,Division,Type,Name,lat,long,Ward,lat_ward,lon_ward,newName,Party,...,Manpower,Finance,PMO,Law,National Development,Home Affairs,Age_group,X,Y,code
0,MARSILING-YEW TEE,GRC,Alex Yam,103.765727,1.423225,Yew Tee,103.749244,1.421106,Alex_Yam,People's Action Party,...,0,0,0,0,0,0,35 - 40,-1.591435,-0.083565,0000000000000000
1,TANJONG PAGAR,GRC,Alvin Tan,103.823059,1.301685,Moulmein-Cairnhill,103.830856,1.313360,Alvin_Tan,People's Action Party,...,0,0,0,0,0,0,35 - 40,2.881252,4.744565,0000000101000000
2,HONG KAH NORTH,SMC,Amy Khor,103.731972,1.362757,HONG KAH NORTH SMC,103.731931,1.362799,Amy_Khor,People's Action Party,...,0,0,0,0,0,0,60 - 65,-5.380332,6.376689,0001100000000000
3,WEST COAST,GRC,Ang Wei Neng,103.705961,1.289002,Nanyang,103.683154,1.344250,Ang_Wei_Neng,People's Action Party,...,0,0,0,0,0,0,50 - 55,-0.956408,0.562835,0000000000000000
4,TAMPINES,GRC,Baey Yam Keng,103.941521,1.359438,Tampines North,103.939705,1.367112,Baey_Yam_Keng,People's Action Party,...,0,0,0,0,0,0,45 - 50,1.023897,-3.111347,0001000000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,JURONG,GRC,Xie Yao Quan,103.745311,1.335105,Jurong Central,103.728056,1.349966,Xie_Yao_Quan,People's Action Party,...,0,0,0,0,0,0,Less than 35,-0.794061,3.126730,0000000000000000
89,PASIR RIS-PUNGGOL,GRC,Yeo Wan Ling,103.931336,1.389701,Punggol North,103.912860,1.396385,Yeo_Wan_Ling,People's Action Party,...,0,0,0,0,0,0,40 - 45,-0.652099,5.721021,0000000000000000
90,YIO CHU KANG,SMC,Yip Hon Weng,103.838759,1.383981,YIO CHU KANG SMC,103.838735,1.384061,Yip_Hon_Weng,People's Action Party,...,0,0,0,0,0,0,40 - 45,0.239466,2.619351,0000000000000000
91,MARSILING-YEW TEE,GRC,Zaqy Mohamad,103.765727,1.423225,Marsiling,103.773119,1.438603,Zaqy_Mohamad,People's Action Party,...,1,0,0,0,0,0,40 - 45,3.197943,0.113861,1000000000100000
